## Spotify Tracks (By Genre)

There are a few columns such as "Valence" (I changed this to "mood") and "Danceability", these all have been [defined here.](https://www.spotify-song-stats.com/about)

In [1]:
%%html
<style>
    table{float:left;}
</style>

#### American Standard Pitch Notation

Number | Pitch  | Number | Pitch  
------ | ------ | ------ | ------
0      | C      | 6      | F♯/G♭
1      | C♯/D♭  | 7      | G    
2      | D      | 8      | A♭/G♯
3      |E♭/D♯   | 9      | A    
4      | E      | 10     | B♭/A♯
5      | F      | 11     | B   

## Imports

In [2]:
# Install the datasets module from HuggingFace.
#!pip install datasets

from datasets import load_dataset
import pandas as pd
import numpy as np

dataset = load_dataset("maharshipandya/spotify-tracks-dataset", data_files="dataset.csv", split="train")

df = pd.DataFrame(dataset)

In [3]:
# memory usage: 17.5+ MB
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

## Cleaning Up Columns

In [4]:
# Dropping columns I don't need.
cols_to_drop = [
        'Unnamed: 0',
        'track_id',
        'liveness',
        'instrumentalness',
        'acousticness',
        'mode',
        'time_signature'
        ]
    
df = df.drop(cols_to_drop, axis=1)

In [5]:
# Columns to uppercase.
df.columns = df.columns.str.upper()

# Renaming to "DURATION" since I'll be converting the ms to HH:MM:SS (or just MM:SS if there aren't any over an hour).
df.rename(columns={'DURATION_MS': 'DURATION'}, inplace=True)

In [6]:
df.head()

,ARTISTS,ALBUM_NAME,TRACK_NAME,POPULARITY,DURATION,EXPLICIT,DANCEABILITY,ENERGY,KEY,LOUDNESS,SPEECHINESS,VALENCE,TEMPO,TRACK_GENRE
0,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0.1430,0.715,87.917,acoustic
1,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,0.0763,0.267,77.489,acoustic
2,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,0.0557,0.120,76.332,acoustic
3,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,0.0363,0.143,181.740,acoustic
4,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,0.0526,0.167,119.949,acoustic


## Data Manipulation / Formatting

In [7]:
# If the duration is less than an hour, it will exclude the HH portion and leave the MM:SS. Otherwise it includes HH.
def trim_hours(value):
    if value.startswith('00:'):
        return value[3:]
    else:
        return value

# Create a function to map valence to the mood. 
def map_valence_to_mood(valence):
    for i, (lower, upper) in enumerate(valence_ranges):
        if lower <= valence <= upper:
            return moods[i]

# Create a function to map valence to the mood. 
def map_energy_to_energy_lvl(energy):
    for i, (lower, upper) in enumerate(energy_ranges):
        if lower <= energy <= upper:
            return energy_levels[i]

key_dict = {
    '0':'C',
    '1':'C♯/D♭',
    '2':'D',
    '3':'E♭/D♯',
    '4':'E',
    '5':'F',
    '6':'F♯/G♭',
    '7':'G',
    '8':'A♭/G♯',
    '9':'A',
    '10':'B♭/A♯',
    '11':'B'
}

# Defining a range of valence and energy for every mood / energy level.
valence_ranges = [(0.0, 0.099), (0.1, 0.199), (0.2, 0.299), (0.3, 0.399), (0.4, 0.499),
          (0.5, 0.599), (0.6, 0.699), (0.7, 0.799), (0.8, 0.899), (0.9, 0.995)]

energy_ranges = [(0.0, 0.099), (0.1, 0.199), (0.2, 0.299), (0.3, 0.399), (0.4, 0.499),
                 (0.5, 0.599), (0.6, 0.699), (0.7, 0.799), (0.8, 0.899), (0.9, 1.0)]

# This isn't too necessary, but wanted to try something like this by turning numerical values into more categorical elements.
# NOTE: These may not be 100% accurate.
moods = ['Dark', 'Melancholic', 'Calm', 'Neutral', 'Positive',
                'Energetic', 'Upbeat', 'Cheerful', 'Uplifting', 'Blissful']

energy_levels = ['Low', 'Low-Mid', 'Mid', 'Mid-High', 'High-Mid',
                 'High', 'Very High', 'Extremely High', 'Maximum', 'Maximum+']

# Converting the duration column from milliseconds to minutes and seconds.
df['DURATION'] = pd.to_datetime(df['DURATION'], unit='ms').dt.strftime('%H:%M:%S')
df['DURATION'] = df['DURATION'].astype("string")
df['DURATION'] = df['DURATION'].apply(trim_hours)

# Apply the function to the 'VALENCE' col and assign the new value to a new 'MOOD' column.
df['MOOD'] = df['VALENCE'].apply(map_valence_to_mood)
df = df.drop(columns='VALENCE', axis=1)

# Similar to the Valence / Mood stuff.
df['ENERGY_LVL'] = df['ENERGY'].apply(map_energy_to_energy_lvl)
df = df.drop(columns='ENERGY', axis=1)

# Replacing the 0 - 11 value in the KEY col to it's actual musical pitch notation. 
df['KEY'] = df['KEY'].astype("string")
df['KEY'] = df['KEY'].map(key_dict)

df['TEMPO'] = df['TEMPO'].astype(int)

df['LOUDNESS'] = np.round(df['LOUDNESS'], decimals=2)

# Saving a bit of repeated code. This is just checking the columns datatype and converting to a smaller size. objects turn into strings.
df = df.apply(lambda x: np.float32(x) if x.dtype == 'float64' else (np.int16(x) if x.dtype == 'int64' else (x.astype("string") if x.dtype == 'object' else x)))

In [8]:
df.head()

,ARTISTS,ALBUM_NAME,TRACK_NAME,POPULARITY,DURATION,EXPLICIT,DANCEABILITY,KEY,LOUDNESS,SPEECHINESS,TEMPO,TRACK_GENRE,MOOD,ENERGY_LVL
0,Gen Hoshino,Comedy,Comedy,73,03:50,False,0.676,C♯/D♭,-6.75,0.1430,87,acoustic,Cheerful,High-Mid
1,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,02:29,False,0.420,C♯/D♭,-17.24,0.0763,77,acoustic,Calm,Low-Mid
2,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,03:30,False,0.438,C,-9.73,0.0557,76,acoustic,Melancholic,Mid-High
3,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,03:21,False,0.266,C,-18.52,0.0363,181,acoustic,Melancholic,Low
4,Chord Overstreet,Hold On,Hold On,82,03:18,False,0.618,D,-9.68,0.0526,119,acoustic,Melancholic,High-Mid


In [9]:
# memory usage: 9.0 MB
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ARTISTS       113999 non-null  string 
 1   ALBUM_NAME    113999 non-null  string 
 2   TRACK_NAME    113999 non-null  string 
 3   POPULARITY    114000 non-null  int16  
 4   DURATION      114000 non-null  string 
 5   EXPLICIT      114000 non-null  bool   
 6   DANCEABILITY  114000 non-null  float32
 7   KEY           114000 non-null  string 
 8   LOUDNESS      114000 non-null  float32
 9   SPEECHINESS   114000 non-null  float32
 10  TEMPO         114000 non-null  int32  
 11  TRACK_GENRE   114000 non-null  string 
 12  MOOD          113911 non-null  string 
 13  ENERGY_LVL    113974 non-null  string 
dtypes: bool(1), float32(3), int16(1), int32(1), string(8)
memory usage: 9.0 MB


In [10]:
df.to_excel('spotify-tracks-by-genre.xlsx')

In [ ]:
'''
new file size (xlsx): 13.4 MB
original (csv): 20.1 MB
''' 